####Load Google Drive File

In [8]:
## Load Folder Drive
import os, sys
import os.path
from os import path
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DATA_UM


# **SECTION FOR DATA SCRAPING**

In [9]:
import pandas as pd
import json
import csv

# dframe = pd.read_csv('data_scraped/'+hashtag+'.csv')

# **SECTION FOR DATA CLEANING**

####1. Data Transformation (Old Process) - Updated Data Transformasi di Collab yg berbeda di 1.2 beda tiap platform. Disini hanya tinggal load. (Do not delete cell)

In [10]:
import re

Load Data Twitter dan Tiktok sisanya menyusul.

In [11]:
df_clean = pd.read_excel(open('data/label/NOT_FINAL_IG.xlsx', 'rb'),sheet_name='dataset')
df_clean.reset_index(drop=True, inplace=True)
df_clean = df_clean.loc[:, ~df_clean.columns.str.contains('^Unnamed')]
df_clean

,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.00
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.00
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.00
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.00
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24189,24190,18008796344074431,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Waiting For 26 January,...,NaN,NaN,NaN,NaN,9,https://www.instagram.com/p/CzFsyLArKoe,Kapann tanggalnya minnn????,not labelled,Language.INDONESIAN,1.00
24190,24191,18010887568845092,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,f,...,NaN,NaN,NaN,NaN,96,https://www.instagram.com/p/CzFsyLArKoe,ada photocard ny ga minn,not labelled,NaN,436.00
24191,24192,18318510622129004,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Mal-,...,NaN,NaN,NaN,NaN,35,https://www.instagram.com/p/CzFsyLArKoe,"Gercep loh min ini min, ayo spill tanggal 😭",not labelled,Language.INDONESIAN,0.88
24192,24193,18203292592279142,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Selly,...,NaN,NaN,NaN,NaN,1,https://www.instagram.com/p/CzFsyLArKoe,Mau lino @fhliu2611,not labelled,NaN,491.00


####2. Text Transformation
1. Delete Punctuations,
2. Delete Number,
3. Delete Link Website,
4. Delete Hashtag,
5. Transform to Lowercase,
6. Replace newline with space,
7. Delete Repeated Char,
8. Delete single word (moved to after stopword), dan
9. Replace double space.

In [12]:
# Import Lib String
import string

In [13]:
# Before Remove Punctuation
df_clean.head(10)

,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.0
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.0
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.0
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.0
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.0
5,6,18007324945447364,2022-07-18,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Annisa Desrizal,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku suka yg mangga sama blueberry min😍,2,Language.INDONESIAN,955.0
6,7,17936245436251405,2022-07-13,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Dian Ekawati Pemalang II,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,@bessedilla hadir tim ketan hitam😍,2,Language.INDONESIAN,977.0
7,8,17947540016064899,2022-07-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Ahmad Bahtiar,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Mimin Endrose akuuuu dong Cimory Squeeze Taro ...,2,NaN,413.0
8,9,17965757539650619,2022-07-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,lani anggraeni,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku ketagihan cimory yogurt squeeze pliss enak...,2,Language.INDONESIAN,1.0
9,10,17936865425224911,2022-07-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Siti Haniza,...,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku keisni gara gara kepo apa yg diminta kianoo,1,Language.INDONESIAN,932.0


In [14]:
def remove_punct(text):
    text = str(text)
    # Remove hashtag dan @
    ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    # Change "-" to space
    text = text.replace("-", " ")

    # Remove punctiation
    text  = "".join([char for char in text if char not in string.punctuation])

    # Remove angka
    text = re.sub('[0-9]+', '', text)

    # Remove website link
    text = re.sub(r'http\S+', '', text)

    # Remove new line
    text = re.sub("\n"," ",text)

    # Transform to lowercase
    text = text.lower()

    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)

    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)

    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    return text

## Running
df_clean['text_punt'] = df_clean['text'].apply(lambda x: remove_punct(x))

## Show Data
df_clean.head(10)

,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score,text_punt
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.0,strawberry kesukaan ku 😍😍😍😍😍
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.0,min gemoy kapan bikin yuk ga produk cimory squ...
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.0,😍❤️ ikutan terbang dong
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.0,pas banget aku lg minum cimory yogurt rasa str...
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.0,pengen beli tapi gaada di malaysia 😭
5,6,18007324945447364,2022-07-18,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Annisa Desrizal,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku suka yg mangga sama blueberry min😍,2,Language.INDONESIAN,955.0,aku suka yg mangga sama blueberry min😍
6,7,17936245436251405,2022-07-13,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Dian Ekawati Pemalang II,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,@bessedilla hadir tim ketan hitam😍,2,Language.INDONESIAN,977.0,bessedilla hadir tim ketan hitam😍
7,8,17947540016064899,2022-07-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Ahmad Bahtiar,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Mimin Endrose akuuuu dong Cimory Squeeze Taro ...,2,NaN,413.0,mimin endrose aku dong cimory squeeze taro nya...
8,9,17965757539650619,2022-07-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,lani anggraeni,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku ketagihan cimory yogurt squeeze pliss enak...,2,Language.INDONESIAN,1.0,aku ketagihan cimory yogurt squeeze pliss enak...
9,10,17936865425224911,2022-07-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Siti Haniza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Aku keisni gara gara kepo apa yg diminta kianoo,1,Language.INDONESIAN,932.0,aku keisni gara gara kepo apa yg diminta kianoo


####3. Penghapusan Kata Singkatan, Slang, Alay, DLL

In [15]:
import string

In [16]:
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')

In [17]:
## Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

## Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

## Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)

<ipython-input-17-97e6cbf1b797>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [18]:
############### Filter Slang awal (20 Menit)
# %%time

# def remove_slang(text):
#     # To Lowercase
#     text = " "+text.lower()+" "
#     for index, row in df_filter_slang.iterrows():
#       # Change Slang to Formal
#       # print(row['slang'], row['formal'])
#       # text = re.sub(r'\b%s\b' % row['slang'], row['formal'], text)
#       text = text.replace(' '+row['slang']+' ', ' '+row['formal']+' ')

#     text = text[1:]
#     text = text.rstrip(text[-1])
#     return text

# ## Testing
# #x = "yang yg dapat dpt blh boleh okay oke"
# #print(remove_slang(x))

# df_clean['text_filter'] = df_clean['text'].apply(lambda x: remove_slang(x))
# df_clean.head(10)

In [19]:
import numpy as np

In [20]:
%%time

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

## Testing
#x = "yang yg dapat dpt blh boleh okay oke"
#print(remove_slang(x))

## Running
df_clean['text_filter1'] = df_clean['text_punt'].apply(lambda x: remove_slang(x,df_slang_nasalsabila))
df_clean['text_filter2'] = df_clean['text_filter1'].apply(lambda x: remove_slang(x,df_slang_fendiirfan))
df_clean['text_filter'] = df_clean['text_filter2'].apply(lambda x: remove_slang(x,df_slang_custom))

## Delete Unused Column
df_clean.drop(columns=['text_punt', 'text_filter1', 'text_filter2'], inplace=True)

## Show Data
df_clean

CPU times: user 43.1 s, sys: 1.42 s, total: 44.6 s
Wall time: 44.2 s


,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score,text_filter
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.00,strawberry kesukaan aku 😍😍😍😍😍
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.00,admin gemoy kapan buat yuk tidak produk cimory...
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.00,😍❤️ ikutan terbang dong
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.00,pas banget saya lagi minum cimory yogurt rasa ...
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.00,pengin beli tapi tidak ada di malaysia 😭
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24189,24190,18008796344074431,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Waiting For 26 January,...,NaN,NaN,NaN,9,https://www.instagram.com/p/CzFsyLArKoe,Kapann tanggalnya minnn????,not labelled,Language.INDONESIAN,1.00,kapann tanggalnya admin
24190,24191,18010887568845092,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,f,...,NaN,NaN,NaN,96,https://www.instagram.com/p/CzFsyLArKoe,ada photocard ny ga minn,not labelled,NaN,436.00,ada photocard nya tidak minn
24191,24192,18318510622129004,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Mal-,...,NaN,NaN,NaN,35,https://www.instagram.com/p/CzFsyLArKoe,"Gercep loh min ini min, ayo spill tanggal 😭",not labelled,Language.INDONESIAN,0.88,gercep loh admin ini admin ayo spill tanggal 😭
24192,24193,18203292592279142,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Selly,...,NaN,NaN,NaN,1,https://www.instagram.com/p/CzFsyLArKoe,Mau lino @fhliu2611,not labelled,NaN,491.00,mau lino fhliu


####4. Stemming Sentence

In [33]:
## Donwload Library Sastrawi for Stemming and Stopword
!pip install Sastrawi

In [34]:
# Import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [23]:
%%time

def stemming_sentence(text):
    text = stemmer.stem(text)
    return text

## Running
df_clean['text_stemming'] = df_clean['text_filter'].apply(lambda x: stemming_sentence(x))

## Delete Unused Column
df_clean.drop(columns=['text_filter'], inplace=True)

## Show Data
df_clean

CPU times: user 42min 43s, sys: 4.67 s, total: 42min 48s
Wall time: 43min 13s


,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score,text_stemming
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.00,strawberry suka aku
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.00,admin gemoy kapan buat yuk tidak produk cimory...
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.00,ikut terbang dong
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.00,pas banget saya lagi minum cimory yogurt rasa ...
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.00,pengin beli tapi tidak ada di malaysia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24189,24190,18008796344074431,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Waiting For 26 January,...,NaN,NaN,NaN,9,https://www.instagram.com/p/CzFsyLArKoe,Kapann tanggalnya minnn????,not labelled,Language.INDONESIAN,1.00,kapann tanggal admin
24190,24191,18010887568845092,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,f,...,NaN,NaN,NaN,96,https://www.instagram.com/p/CzFsyLArKoe,ada photocard ny ga minn,not labelled,NaN,436.00,ada photocard nya tidak minn
24191,24192,18318510622129004,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Mal-,...,NaN,NaN,NaN,35,https://www.instagram.com/p/CzFsyLArKoe,"Gercep loh min ini min, ayo spill tanggal 😭",not labelled,Language.INDONESIAN,0.88,gercep loh admin ini admin ayo spill tanggal
24192,24193,18203292592279142,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Selly,...,NaN,NaN,NaN,1,https://www.instagram.com/p/CzFsyLArKoe,Mau lino @fhliu2611,not labelled,NaN,491.00,mau lino fhliu


#### 5. Remove Stopwords

In [24]:
## Load Library Stopword for Indonesian Language Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [25]:
# Load Data Stopword and Custom Word
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])
keep_custom_stopword = list(pd.read_csv('data/stopword/keep_custom_stopword.txt', header = None)[0])

In [26]:
def remove_stopwords(text):
  # Removal Stopword
  text_split = text.split(' ')

  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      ## Keep Beberapa Kata Valence of a text
      # https://datascience.stackexchange.com/questions/15765/nlp-why-is-not-a-stop-word#:~:text=Stop%20words%20are%20usually%20thought,idf%20analysis%20for%20document%20classification).
      if (text_split[i] in keep_custom_stopword) :
        temp_text_split.append(text_split[i])
      else :
        ## Stopword Removal from data by Sastrawi
        tmp = stopword.remove(text_split[i])
        temp_text_split.append(tmp)

  final_text = ' '.join(temp_text_split)

  # will replace single word
  final_text = re.sub(r"\b[a-zA-Z]\b","",final_text)

  # will replace space more than one
  final_text = re.sub("( {2,})"," ",final_text)

  return final_text

## Running
df_clean['text_final'] = df_clean['text_stemming'].apply(lambda x: remove_stopwords(x))

## Delete Unused Column
df_clean.drop(columns=['text_stemming'], inplace=True)

## Show Data
df_clean

,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score,text_final
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.00,strawberry suka aku
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.00,admin gemoy kapan buat tidak produk cimory squ...
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.00,ikut terbang dong
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.00,pas banget minum cimory yogurt rasa strawberry
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.00,pengin beli tidak malaysia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24189,24190,18008796344074431,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Waiting For 26 January,...,NaN,NaN,NaN,9,https://www.instagram.com/p/CzFsyLArKoe,Kapann tanggalnya minnn????,not labelled,Language.INDONESIAN,1.00,kapann tanggal admin
24190,24191,18010887568845092,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,f,...,NaN,NaN,NaN,96,https://www.instagram.com/p/CzFsyLArKoe,ada photocard ny ga minn,not labelled,NaN,436.00,photocard tidak minn
24191,24192,18318510622129004,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Mal-,...,NaN,NaN,NaN,35,https://www.instagram.com/p/CzFsyLArKoe,"Gercep loh min ini min, ayo spill tanggal 😭",not labelled,Language.INDONESIAN,0.88,gercep loh admin admin ayo spill tanggal
24192,24193,18203292592279142,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Selly,...,NaN,NaN,NaN,1,https://www.instagram.com/p/CzFsyLArKoe,Mau lino @fhliu2611,not labelled,NaN,491.00,mau lino fhliu


####6. Tokenization (Commented : Belum kepake di proses sampai saat ini)

In [27]:
# df_clean
# df_clean.to_csv('data_scraped/data_ok.csv')
# df_clean.to_excel('data_scraped/data_ok.xlsx')
# df_clean

In [28]:
df_clean

,,id,date,platform,brand,comment type,post type,post owner,hashtag,name,...,share_retweet,bookmark,quotes,likes,link,text,labelling,lang,conf_score,text_final
0,1,17932892990317232,2022-09-10,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Umi Faza,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Strawberry kesukaan ku 😍😍😍😍😍,2,Language.INDONESIAN,883.00,strawberry suka aku
1,2,17956733380953641,2022-08-19,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,ridha fitria b,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Minnn gemoyyy kapan2 bikin yuk GA produk cimor...,1,Language.INDONESIAN,605.00,admin gemoy kapan buat tidak produk cimory squ...
2,3,17983198912575017,2022-08-09,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fey Fy,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,😍❤️ ikutan terbang dong..,1,Language.INDONESIAN,976.00,ikut terbang dong
3,4,17856856355789804,2022-08-07,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,𝒓𝒆𝒚ఌ︎,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pas banget aku lg minum cimory yogurt rasa str...,2,NaN,438.00,pas banget minum cimory yogurt rasa strawberry
4,5,17872412072737531,2022-08-04,Instagram,cimory,non-official,official,cimoryindonesia,#CimoryYogurt,Fasya,...,NaN,NaN,NaN,0,https://www.instagram.com/p/Cfd15Wmp6PY,Pengen beli tapi gaada di Malaysia 😭,2,Language.INDONESIAN,978.00,pengin beli tidak malaysia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24189,24190,18008796344074431,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Waiting For 26 January,...,NaN,NaN,NaN,9,https://www.instagram.com/p/CzFsyLArKoe,Kapann tanggalnya minnn????,not labelled,Language.INDONESIAN,1.00,kapann tanggal admin
24190,24191,18010887568845092,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,f,...,NaN,NaN,NaN,96,https://www.instagram.com/p/CzFsyLArKoe,ada photocard ny ga minn,not labelled,NaN,436.00,photocard tidak minn
24191,24192,18318510622129004,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Mal-,...,NaN,NaN,NaN,35,https://www.instagram.com/p/CzFsyLArKoe,"Gercep loh min ini min, ayo spill tanggal 😭",not labelled,Language.INDONESIAN,0.88,gercep loh admin admin ayo spill tanggal
24192,24193,18203292592279142,2023-11-01,Instagram,ultramilk,non-official,official,ultra_mym,#iamULTRAoddinary,Selly,...,NaN,NaN,NaN,1,https://www.instagram.com/p/CzFsyLArKoe,Mau lino @fhliu2611,not labelled,NaN,491.00,mau lino fhliu


In [29]:
# Download Library NLTK untuk Tokenizernya.
!pip install nltk

# Import Lib NLTK dan Tokenizer
import nltk
nltk.download("popular")
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
# def tokenization(text):
#     word_token = word_tokenize(text)
#     return word_token

# df_clean['text_token'] = df_clean['text_stemming'].apply(lambda x: tokenization(x.lower()))
# df_clean.head(10)

#### [LAST] Save Data Checkpoint CSV

In [ ]:
## Duplicate Dataframe
df_final = df_clean

## Delete NULL value in data after preprocessing
# Replace Data null to nan
df_final.replace(' ', np.nan, inplace=True)
df_final.replace('', np.nan, inplace=True)

# Delete Data nan
df_final.dropna(subset=['text_final'], inplace=True)

## Preview Data
df_final

In [32]:
platform = 'ig'

# Check Path is Available if no Create Dir
if path.exists(root_file+'/data/preprocess/'+platform) == False:
  os.makedirs(root_file+'/data/preprocess/'+platform)
else :
  print("File Already Existed")

# Export the Data
df_final.to_csv('data/preprocess/'+platform+'.csv', encoding='utf-8', index=False)
df_final.to_excel('data/preprocess/'+platform+'.xlsx', encoding='utf-8', index=False)

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
